In [1]:
from vectorbtpro import *
from dune_client.client import DuneClient
import fedfred as fd

import tzlocal
import pytz

# Data Fetching

In this notebook we fetch the data necessary for the ML strategies, and technical analysis (TA) strategies. We use third-party data providers, with which you will have to sign up and register keys (for free).

* [Dune Analytics](https://docs.dune.com/api-reference/overview/authentication)
* [Federal Reserve Bank of St. Louis (FRED)](https://fred.stlouisfed.org/docs/api/api_key.html)

Then create a file in the current directory named `.env` with the following:
```
DUNE_API_KEY=<key>
FRED_API_KEY=<key>
```

In [2]:
tzlocal.get_localzone = lambda: pytz.timezone('UTC')

In [3]:
# retrieve API keys from environment
api_keys = dict()
with open(".env") as f:
    for line in f.readlines():
        elems = line.split("=")
        api_keys[elems[0].strip()] = elems[1].strip()
api_keys

{'DUNE_API_KEY': 'hafQP5vPw8oF8qovHAiwbLTbxpdN95gW',
 'FRED_API_KEY': '3a535693a7c1bb047269acdb2e4d7a0a'}

In [4]:
dune = DuneClient(api_key=api_keys['DUNE_API_KEY'])
fred = fd.FredAPI(api_key=api_keys['FRED_API_KEY'])

In [5]:
START_DATE = "2024-04-01"
END_DATE = "2025-12-18"
res_df = pd.DataFrame(index=pd.date_range(START_DATE, END_DATE, tz='UTC'))

The goal is to add useful features to our model: variables that could impact price performance of SOLUSD. For example the [M2 money supply](https://fred.stlouisfed.org/series/M2SL) macro-economic indicator could be a way to track QE which impacts BTC and SOL.

In [6]:
for tix in ["SPY", "NDX", "^VIX", "QQQ", "BTC-USD", "SOL-USD"]:
    data = vbt.YFData.pull(
        tix,
        start=START_DATE,
        end=END_DATE,
        timeframe="1 day",
        tz="UTC"
    )
    df = data.get()
    df.index = pd.to_datetime(df.index, utc=True).normalize()
    res_df = res_df.join(df, lsuffix=f"_{tix}")

In [7]:
for tix in ['DGS10','GDP','FEDFUNDS','WALCL','M2V','M2SL', 'DTWEXBGS']:
    b10 = fred.get_series_observations(tix)
    ser = b10['value'][START_DATE:]
    ser.index = pd.to_datetime(ser.index, utc=True).normalize()
    res_df[tix] = ser
    
res_df = res_df.ffill() # forward-fill infrequently-observed timeseries

In [8]:
query_result = dune.get_latest_result(3930956)
metrics_df = pd.DataFrame(query_result.result.rows)
metrics_df.set_index('time_period', inplace=True)
metrics_df.index = pd.to_datetime(metrics_df.index, utc=True).normalize()
res_df = res_df.join(metrics_df)

In [9]:
res_df.columns

Index(['Open_NDX', 'High_NDX', 'Low_NDX', 'Close_NDX', 'Volume_NDX',
       'Dividends_NDX', 'Stock Splits_NDX', 'Capital Gains_QQQ', 'Open_^VIX',
       'High_^VIX', 'Low_^VIX', 'Close_^VIX', 'Volume_^VIX', 'Dividends_^VIX',
       'Stock Splits_^VIX', 'Open_QQQ', 'High_QQQ', 'Low_QQQ', 'Close_QQQ',
       'Volume_QQQ', 'Dividends_QQQ', 'Stock Splits_QQQ', 'Open_BTC-USD',
       'High_BTC-USD', 'Low_BTC-USD', 'Close_BTC-USD', 'Volume_BTC-USD',
       'Dividends_BTC-USD', 'Stock Splits_BTC-USD', 'Capital Gains',
       'Open_SOL-USD', 'High_SOL-USD', 'Low_SOL-USD', 'Close_SOL-USD',
       'Volume_SOL-USD', 'Dividends_SOL-USD', 'Stock Splits_SOL-USD', 'Open',
       'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'DGS10',
       'GDP', 'FEDFUNDS', 'WALCL', 'M2V', 'M2SL', 'DTWEXBGS',
       'market_capitalization', 'mvrv', 'nupl', 'price',
       'realized_capitalization', 'realized_price', 'supply'],
      dtype='str')

In [10]:
res_df

,Open_NDX,High_NDX,Low_NDX,Close_NDX,Volume_NDX,Dividends_NDX,Stock Splits_NDX,Capital Gains_QQQ,Open_^VIX,High_^VIX,...,M2V,M2SL,DTWEXBGS,market_capitalization,mvrv,nupl,price,realized_capitalization,realized_price,supply
2024-04-01 00:00:00+00:00,512.780997,513.319384,509.981277,511.146179,62477500.0,0.0,0.0,0.0,18293.199219,18293.199219,...,1.389,20929.1,121.5151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-02 00:00:00+00:00,507.308982,508.033364,505.586095,507.896362,74230300.0,0.0,0.0,0.0,18121.779297,18121.779297,...,1.389,20929.1,121.3035,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-03 00:00:00+00:00,506.799837,509.961749,506.750904,508.454193,59155800.0,0.0,0.0,0.0,18160.189453,18160.189453,...,1.389,20929.1,121.0198,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-04 00:00:00+00:00,512.477579,512.820173,501.944527,502.247986,96858100.0,0.0,0.0,0.0,17878.779297,17878.779297,...,1.389,20929.1,120.7613,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-05 00:00:00+00:00,503.608731,509.462578,503.168211,507.494965,74546500.0,0.0,0.0,0.0,18108.460938,18108.460938,...,1.389,20929.1,120.9646,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-14 00:00:00+00:00,686.142492,686.850422,677.169008,679.751404,113160300.0,0.0,0.0,0.0,25196.730469,25196.730469,...,1.406,22411.0,120.1442,3.907911e+08,0.510863,-0.957474,0.391186,7.649634e+08,0.765736,9.989910e+08
2025-12-15 00:00:00+00:00,683.719676,683.739636,677.248806,678.724426,90811000.0,0.0,0.0,0.0,25067.259766,25067.259766,...,1.406,22411.0,119.9561,3.835396e+08,0.505826,-0.976964,0.383927,7.582439e+08,0.759010,9.989910e+08
2025-12-16 00:00:00+00:00,677.228859,679.073445,672.991380,676.869934,122030600.0,0.0,0.0,0.0,25132.939453,25132.939453,...,1.406,22411.0,119.8902,3.710720e+08,0.491650,-1.033967,0.371447,7.547481e+08,0.755510,9.989910e+08
2025-12-17 00:00:00+00:00,677.886913,678.435280,669.222513,669.421936,110625200.0,0.0,0.0,0.0,24647.609375,24647.609375,...,1.406,22411.0,120.1550,3.716993e+08,0.490334,-1.039425,0.372075,7.580528e+08,0.758818,9.989910e+08


In [11]:
res_df.to_csv("data/dataset1.csv", index_label="timestamp")